In [1]:
import torch
import torchvision
from torch import nn, optim

from torchsummary import summary

In [2]:
batch_size = 32
epoch = 30
learning_rate = 0.01

In [3]:
trans = torchvision.transforms.ToTensor()

train_data = torch.utils.data.DataLoader(
            torchvision.datasets.MNIST(
            'mnist_data', train=True, download=True, transform=trans
            ), batch_size=batch_size
            )
val_data = torch.utils.data.DataLoader(
            torchvision.datasets.MNIST(
            'mnist_data', train=False, download=True, transform=trans
            ), batch_size=batch_size)


Extracting mnist_data\MNIST\raw\train-images-idx3-ubyte.gz to mnist_data\MNIST\raw




Extracting mnist_data\MNIST\raw\train-labels-idx1-ubyte.gz to mnist_data\MNIST\raw




Extracting mnist_data\MNIST\raw\t10k-images-idx3-ubyte.gz to mnist_data\MNIST\raw




Extracting mnist_data\MNIST\raw\t10k-labels-idx1-ubyte.gz to mnist_data\MNIST\raw



In [4]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3, stride=1)
        self.maxpool1 = nn.MaxPool2d((2,2))
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=1)
        self.maxpool2 = nn.MaxPool2d((2,2))
        
        self.relu = nn.ReLU()
        self.linear1 = nn.Linear(150, 10)

    def forward(self, x):
        x = self.maxpool1(self.relu(self.conv1(x)))
        x = self.maxpool2(self.relu(self.conv2(x)))
        x = x.view(x.shape[0], -1)
        x = self.linear1(x)
        return x

In [5]:
def validate(model, data):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(data):
        y_pred = model(images)
        value, pred = torch.max(y_pred, 1)
        total += y_pred.size(0)
        correct += torch.sum(pred == labels)
    return correct * 100 / total

In [6]:
convnet = ConvNet()
summary(convnet, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 3, 26, 26]              30
              ReLU-2            [-1, 3, 26, 26]               0
         MaxPool2d-3            [-1, 3, 13, 13]               0
            Conv2d-4            [-1, 6, 11, 11]             168
              ReLU-5            [-1, 6, 11, 11]               0
         MaxPool2d-6              [-1, 6, 5, 5]               0
            Linear-7                   [-1, 10]           1,510
Total params: 1,708
Trainable params: 1,708
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.01
Estimated Total Size (MB): 0.06
----------------------------------------------------------------


In [7]:
%%time

optimizer = optim.Adam(convnet.parameters(), lr=learning_rate)
cross_entropy = nn.CrossEntropyLoss()

for n in range(epoch):
    for i, (images, labels) in enumerate(train_data):
        optimizer.zero_grad()
        prediction = convnet(images)
        loss = cross_entropy(prediction, labels)
        loss.backward()
        optimizer.step()
    accuracy = float(validate(convnet, val_data))
    print("Epoch:", n+1, "Loss: ", float(loss.data), "Val. Accuracy:", accuracy)

Epoch: 1 Loss:  0.05259706452488899 Val. Accuracy: 94.94999694824219
Epoch: 2 Loss:  0.052410438656806946 Val. Accuracy: 94.94000244140625
Epoch: 3 Loss:  0.0477810874581337 Val. Accuracy: 94.76000213623047
Epoch: 4 Loss:  0.038834888488054276 Val. Accuracy: 94.7699966430664
Epoch: 5 Loss:  0.06164509803056717 Val. Accuracy: 94.69999694824219
Epoch: 6 Loss:  0.0478551909327507 Val. Accuracy: 94.0199966430664
Epoch: 7 Loss:  0.08872967213392258 Val. Accuracy: 95.6500015258789
Epoch: 8 Loss:  0.3225192129611969 Val. Accuracy: 95.4000015258789
Epoch: 9 Loss:  0.15344081819057465 Val. Accuracy: 95.95999908447266
Epoch: 10 Loss:  0.223435178399086 Val. Accuracy: 95.08999633789062
Epoch: 11 Loss:  0.1052035242319107 Val. Accuracy: 95.69000244140625
Epoch: 12 Loss:  0.06465202569961548 Val. Accuracy: 95.98999786376953
Epoch: 13 Loss:  0.12556135654449463 Val. Accuracy: 95.2699966430664
Epoch: 14 Loss:  0.010636642575263977 Val. Accuracy: 95.8499984741211
Epoch: 15 Loss:  0.0344492569565773 Va

We can see that ReLU activation function gives better accuracy.